In [1]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path
import os, sys
import numpy as np

import torch
from torch.utils.data import DataLoader, DistributedSampler, random_split, TensorDataset
from torch import nn

In [2]:
split = "val"
model_data_path = "./data/5_scale_31/"
data_path = model_data_path + split + "/data/"
summary_annotation_file = model_data_path + split + "/summary.json"

In [51]:
with open(data_path + "0.json", "r") as outfile:
    data = json.load(outfile)
print(data['input'].keys())
print(data['annotation'].keys())

dict_keys(['pred_logits', 'pred_boxes', 'aux_outputs', 'interm_outputs', 'interm_outputs_for_matching_pre', 'dn_meta'])
dict_keys(['loss', 'loss_bbox_dn', 'loss_giou_dn', 'loss_ce_dn', 'loss_ce', 'loss_bbox', 'loss_giou', 'loss_ce_0', 'loss_bbox_0', 'loss_giou_0', 'loss_bbox_dn_0', 'loss_giou_dn_0', 'loss_ce_dn_0', 'loss_ce_1', 'loss_bbox_1', 'loss_giou_1', 'loss_bbox_dn_1', 'loss_giou_dn_1', 'loss_ce_dn_1', 'loss_ce_2', 'loss_bbox_2', 'loss_giou_2', 'loss_bbox_dn_2', 'loss_giou_dn_2', 'loss_ce_dn_2', 'loss_ce_3', 'loss_bbox_3', 'loss_giou_3', 'loss_bbox_dn_3', 'loss_giou_dn_3', 'loss_ce_dn_3', 'loss_ce_4', 'loss_bbox_4', 'loss_giou_4', 'loss_bbox_dn_4', 'loss_giou_dn_4', 'loss_ce_dn_4', 'loss_ce_interm', 'loss_bbox_interm', 'loss_giou_interm', 'loss_bbox_dn_unscaled', 'loss_giou_dn_unscaled', 'loss_ce_dn_unscaled', 'loss_xy_dn_unscaled', 'loss_hw_dn_unscaled', 'cardinality_error_dn_unscaled', 'loss_ce_unscaled', 'class_error_unscaled', 'loss_bbox_unscaled', 'loss_giou_unscaled', 'loss

In [4]:
# def read_one_key_from_file(file, first_key, second_key, third_key=None):
#     with open(file, "r") as outfile:
#         data = json.load(outfile)
#     return data[first_key][second_key]

In [5]:
# files_num = 5000
# files = os.listdir(data_path)
# assert len(files) == files_num
# inputs = []
# annotations = []
# for file_idx in range(files_num):
#     file_path = data_path + str(file_idx) + ".json"
#     with open(file_path, "r") as outfile:
#         data = json.load(outfile)
#         pred_logits = np.squeeze(np.array(data['input']['pred_logits']), axis=0)
#         pred_boxes = np.squeeze(np.array(data['input']['pred_boxes']), axis=0)
#         pred_input = np.concatenate((pred_logits, pred_boxes), axis=1)
#         inputs.append(pred_input.tolist())
#         loss_ce = data['annotation']['loss_ce']
#         loss_bbox = data['annotation']['loss_bbox']
#         loss_giou = data['annotation']['loss_giou']
#         annotations.append(loss_ce + loss_bbox + loss_giou)
#     if file_idx % 100 == 0:
#         print(file_idx)

In [6]:
# store_preprocess_inputs_path = model_data_path + split + "/pre_data/val_inputs.npy"
# with open(store_preprocess_inputs_path, "wb") as outfile:
#     np.save(outfile, np.array(inputs))

In [7]:
# store_preprocess_annotations_path = model_data_path + split + "/pre_data/val_annotations.npy"
# with open(store_preprocess_annotations_path, "wb") as outfile:
#     np.save(outfile, np.array(annotations))

## model

In [8]:
class MPLNet(nn.Module):
    def __init__(self, input_dims = 10000, output_dims = 1, dropout = 0.1):
        super().__init__()
        self.layer1=nn.Linear(input_dims, 10000)
        self.layer2=nn.Linear(10000, 1000)
        self.layer3=nn.Linear(1000, 100)
        self.layer4=nn.Linear(100, 10)
        self.layer5=nn.Linear(10, 1)
        self.norm1 = nn.LayerNorm(10000)
        self.norm2 = nn.LayerNorm(1000)
        self.norm3 = nn.LayerNorm(100)
        self.norm4 = nn.LayerNorm(10)
        self._init_parms(self.layer1)
        self._init_parms(self.layer2)
        self._init_parms(self.layer3)
        self._init_parms(self.layer4)
        self._init_parms(self.layer5)
        # self.dropout1 = nn.Dropout(dropout)
        # self.dropout2 = nn.Dropout(dropout)
        # self.dropout3 = nn.Dropout(dropout)
        self.input_dims = input_dims
    
    def _init_parms(self, module):
        module.weight.data.normal_(mean=0.0, std=1.0)
        
    def forward(self, x):
        x=x.view(-1, self.input_dims)
        x=nn.functional.relu(self.norm1(self.layer1(x)))
        x=nn.functional.relu(self.norm2(self.layer2(x)))
        x=nn.functional.relu(self.norm3(self.layer3(x)))
        x=nn.functional.relu(self.norm4(self.layer4(x)))
        x=self.layer5(x)
        return x

In [9]:
device_name = "cuda:1"
device = torch.device(device_name)
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
rate_learning = 1e-4
model = MPLNet(input_dims=85500)
loss_function = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=rate_learning)
model.to(device)

MPLNet(
  (layer1): Linear(in_features=85500, out_features=10000, bias=True)
  (layer2): Linear(in_features=10000, out_features=1000, bias=True)
  (layer3): Linear(in_features=1000, out_features=100, bias=True)
  (layer4): Linear(in_features=100, out_features=10, bias=True)
  (layer5): Linear(in_features=10, out_features=1, bias=True)
  (norm1): LayerNorm((10000,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((1000,), eps=1e-05, elementwise_affine=True)
  (norm3): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  (norm4): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
)

In [10]:
# load data
batch_size = 10
store_preprocess_inputs_path = model_data_path + split + "/pre_data/val_inputs.npy"
store_preprocess_annotations_path = model_data_path + split + "/pre_data/val_annotations.npy"
with open(store_preprocess_inputs_path, 'rb') as outfile:
    inputs = torch.from_numpy(np.load(outfile))
with open(store_preprocess_annotations_path, 'rb') as outfile:
    annotations = torch.from_numpy(np.load(outfile))
    
datasets = TensorDataset(inputs, annotations)
datasets_nums = inputs.shape[0]
train_nums = int(datasets_nums * 0.7)
train_datasets, val_datasets = torch.utils.data.random_split(datasets, [train_nums, datasets_nums-train_nums])
train_sampler = torch.utils.data.RandomSampler(train_datasets)
batch_sampler_train = torch.utils.data.BatchSampler(train_sampler, batch_size, drop_last=True)
train_dataloader = DataLoader(train_datasets, batch_sampler=batch_sampler_train)

val_sampler = torch.utils.data.SequentialSampler(val_datasets)
valid_dataloader = DataLoader(val_datasets, 1, sampler=val_sampler, drop_last=False)

In [11]:
for epoch in range(10):

    # Performing Training for each epoch
    training_loss = 0.
    model.train()

    # The training loop
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        input, label = batch
        input = input.to(device=device_name).type(torch.float)
        label = label.to(device=device_name).type(torch.float)[None, :]
        output = model(input)
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        if step % 100 == 0:
            print("Train Epoch: {}, Step: {}, Loss {}".format(epoch, step, loss.item()))


    # Performing Validation for each epoch
    validation_loss = 0.
    model.eval()

    # The validation loop
    for batch in valid_dataloader:
        input, label = batch
        input = input.to(device=device_name).type(torch.float)
        label = label.to(device=device_name).type(torch.float)[None, :]
        output = model(input)
        loss = loss_function(output, label)
        validation_loss += loss.item()

    # Calculating the average training and validation loss over epoch
    training_loss_avg = training_loss/len(train_dataloader)
    validation_loss_avg = validation_loss/len(valid_dataloader)

    # Printing average training and average validation losses
    print("Epoch: {}".format(epoch))
    print("Training loss: {}".format(training_loss_avg))
    print("Validation loss: {}".format(validation_loss_avg))

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train Epoch: 0, Step: 0, Loss 1.4659523963928223
Train Epoch: 0, Step: 100, Loss 0.05399113893508911
Train Epoch: 0, Step: 200, Loss 0.18286354839801788
Train Epoch: 0, Step: 300, Loss 0.06545349955558777
Epoch: 0
Training loss: 0.2372349515557289
Validation loss: 0.15801718249951247
Train Epoch: 1, Step: 0, Loss 0.031024688854813576
Train Epoch: 1, Step: 100, Loss 0.0839051604270935
Train Epoch: 1, Step: 200, Loss 0.31396323442459106
Train Epoch: 1, Step: 300, Loss 0.17906448245048523
Epoch: 1
Training loss: 0.16389689891998258
Validation loss: 0.16586352959494116
Train Epoch: 2, Step: 0, Loss 0.10907010734081268
Train Epoch: 2, Step: 100, Loss 0.18140292167663574
Train Epoch: 2, Step: 200, Loss 0.14571760594844818
Train Epoch: 2, Step: 300, Loss 0.38280653953552246
Epoch: 2
Training loss: 0.16617160986576762
Validation loss: 0.16548182156589106
Train Epoch: 3, Step: 0, Loss 0.11528050899505615
Train Epoch: 3, Step: 100, Loss 0.10774984210729599
Train Epoch: 3, Step: 200, Loss 1.08178

KeyboardInterrupt: 

In [50]:
model.eval()
input = inputs[10].to(device=device_name).type(torch.float)
output = model(input)
output

tensor([[0.6111]], device='cuda:1', grad_fn=<AddmmBackward0>)

In [45]:
torch.save(model.state_dict(), "ckpts/5_scale_31_mpl.pickle")

## Active testing

In [38]:
def LURE_weights_for_risk_estimator(weights, N):
    M = weights.size
    if M < N:
        m = np.arange(1, M+1)
        v = (
            1
            + (N-M)/(N-m) * (
                    1 / ((N-m+1) * weights)
                    - 1
                    )
            )
    else:
        v = 1

    return v

def acquire(expected_loss_inputs, samples_num):
    assert samples_num <= expected_loss_inputs.size
    expected_loss = np.copy(expected_loss_inputs)
    # Log-lik can be negative.
    # Make all values positive.
    if (expected_loss < 0).sum() > 0:
        expected_loss += np.abs(expected_loss.min())
    
    if np.any(np.isnan(expected_loss)):
        logging.warning(
            'Found NaN values in expected loss, replacing with 0.')
        logging.info(f'{expected_loss}')
        expected_loss = np.nan_to_num(expected_loss, nan=0)
    pick_sample_idxs = np.zeros((samples_num), dtype = int)
    idx_array = np.arange(expected_loss.size)
    weights = np.zeros((samples_num), dtype = np.single)
    uniform_clip_val = 0.2
    for i in range(samples_num):
        expected_loss /= expected_loss.sum()
        # clip all values less than 10 percent of uniform propability
        expected_loss = np.maximum(uniform_clip_val * 1/expected_loss.size, expected_loss)
        expected_loss /= expected_loss.sum()
        sample = np.random.multinomial(1, expected_loss)
        cur_idx = np.where(sample)[0][0]
        pick_sample_idxs[i] = idx_array[cur_idx]
        weights[i] = expected_loss[cur_idx]
        selected_mask = np.ones((expected_loss.size), dtype=bool)
        selected_mask[cur_idx] = False
        expected_loss = expected_loss[selected_mask]
        idx_array = idx_array[selected_mask]
    return pick_sample_idxs, weights

def get_expected_loss():
    model.eval()
    datasets_nums = inputs.shape[0]
    expected_loss = np.zeros(datasets_nums)
    for i in range(datasets_nums):
        input = inputs[i].to(device=device_name).type(torch.float)
        output = model(input)
        expected_loss[i] = output.detach().cpu().numpy()
    return expected_loss

def run_one_approximate_risk_estimator(true_losses, expected_losses, seed, samples_num):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    pick_sample_idxs, weights = acquire(expected_losses, samples_num)
    risk_estimator_weights = LURE_weights_for_risk_estimator(weights, expected_losses.size)
    sampled_true_losses = true_losses[pick_sample_idxs]

    loss_risk = (sampled_true_losses * risk_estimator_weights).mean()
    return loss_risk

def get_true_loss():
    return annotations.numpy()

In [35]:
true_losses = get_true_loss()
expected_losses = get_expected_loss()

In [36]:
result_json_path = "./results/MPL_loss_compare/"
sample_size_set = [50, 100, 150, 200, 250, 500, 750, 1000, 1500, 2000, 3000]
random_seed_set = [4519, 9524, 5901, 1028, 6382, 5383, 5095, 7635,  890,  608]

In [39]:
file_path = result_json_path + "MPL_R50_31_10_runs.json"
json_object = {}
for sample_size in sample_size_set:
    for seed in random_seed_set:
        result = {"active_test_type": "MPL", "sample_size": sample_size}
        loss_risk = run_one_approximate_risk_estimator(true_losses, expected_losses, seed, sample_size)
        result["loss"] = loss_risk
        json_object[len(json_object)] = result
with open(file_path, "w") as outfile:
    json.dump(json_object, outfile)

## Random Sample

In [40]:
def run_one_random_sample_risk_estimator(true_losses, seed, samples_num):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    perm = np.random.permutation(true_losses.size)
    pick_sample_idxs = perm[:samples_num]
    sampled_true_losses = true_losses[pick_sample_idxs]
    return sampled_true_losses.mean()

In [41]:
file_path = result_json_path + "random_sample_R50_31_10_runs.json"
json_object = {}
for sample_size in sample_size_set:
    for seed in random_seed_set:
        result = {"active_test_type": "random sample", "sample_size": sample_size}
        loss_risk = run_one_random_sample_risk_estimator(true_losses, seed, sample_size)
        result["loss"] = loss_risk
        json_object[len(json_object)] = result
with open(file_path, "w") as outfile:
    json.dump(json_object, outfile)

In [47]:
def run_one_random_sample_risk_estimator_LURE(true_losses, seed, samples_num):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    perm = np.random.permutation(true_losses.size)
    pick_sample_idxs = perm[:samples_num]
    sampled_true_losses = true_losses[pick_sample_idxs]
    weights = np.ones(samples_num) * 1.0 / samples_num
    risk_estimator_weights = LURE_weights_for_risk_estimator(weights, true_losses.size)
    loss_risk = (sampled_true_losses * risk_estimator_weights).mean()
    return loss_risk

In [48]:
file_path = result_json_path + "random_sample_LURE_R50_31_10_runs.json"
json_object = {}
for sample_size in sample_size_set:
    for seed in random_seed_set:
        result = {"active_test_type": "random sample LURE", "sample_size": sample_size}
        loss_risk = run_one_random_sample_risk_estimator_LURE(true_losses, seed, sample_size)
        result["loss"] = loss_risk
        json_object[len(json_object)] = result
with open(file_path, "w") as outfile:
    json.dump(json_object, outfile)

## Whole data set risk estimation

In [42]:
def get_whole_data_set_risk_estimator(true_losses):
    return true_losses.mean()

In [44]:
file_path = result_json_path + "None_R50_31.json"
result = {"active_test_type": "None", "sample_size": inputs.shape[0]}
result["loss"] = get_whole_data_set_risk_estimator(true_losses)
json_object = {}
json_object[0] = result
with open(file_path, "w") as outfile:
    json.dump(json_object, outfile)